# Predicción de Cantidades Vendidas por Región y Categoría

Este notebook realiza la predicción de cantidades vendidas por región y categoría utilizando XGBoost, considerando variables temporales y otras características relevantes. Los componentes principales incluyen:

- **Carga de datos** desde la vista `vw_ventas_ml` en Snowflake.
- **Preprocesamiento**: Manejo de valores nulos, codificación de variables categóricas, creación de características temporales y escalado de datos.
- **Selección de características**: Uso de Recursive Feature Elimination (RFE) para identificar las características más importantes.
- **Entrenamiento del modelo**: Optimización de hiperparámetros mediante RandomizedSearchCV.
- **Predicciones**: Generación de predicciones para 12 meses futuros con intervalos de confianza del 95% por región y categoría.
- **Métricas**: Cálculo de RMSE, MAE, SMAPE y R² para entrenamiento, prueba y validación.
- **Almacenamiento**: Guardado de predicciones (TRUNCATE + COPY INTO) y métricas (COPY INTO append) en Snowflake.
- **Visualización**: Gráficas interactivas con Plotly para datos históricos y predicciones por región y categoría.

**Objetivo**: Producir pronósticos precisos de cantidades vendidas por región y categoría, considerando descuentos y márgenes de ganancia.

## 1. Importar Librerías y Configuración

Importamos las librerías necesarias y configuramos el logging para trazabilidad.

In [320]:
import pandas as pd
import numpy as np
import snowflake.connector
import logging
from datetime import datetime
import os
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb
import joblib
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings

# Configurar advertencias y logging
warnings.filterwarnings("ignore")
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler()
    ]
)

## 2. Conectar a la Base de Datos (Snowflake)

Establecemos una conexión segura a Snowflake para cargar datos y almacenar resultados.

In [321]:
def get_snowflake_connection():
    try:
        conn = snowflake.connector.connect(
        user='ringoquimicodev',  
        password='Ch3cooch2ch2ch3',  
        account='RSB72105', 
        warehouse='COMPUTE_WH', 
        database='BEBIDAS_PROJECT',
        schema='BEBIDAS_ANALYTICS'
        )
        logging.info("Conexión a Snowflake exitosa")
        return conn
    except Exception as e:
        logging.error(f"Error de conexión a Snowflake: {e}")
        raise

## 3. Cargar y Preprocesar Datos

Cargamos los datos desde la vista `vw_ventas_ml`, manejamos valores nulos, creamos características temporales y codificamos variables categóricas.

In [322]:
def load_and_preprocess_data():
    try:
        conn = get_snowflake_connection()
        query = "SELECT * FROM vw_ventas_ml"
        df = pd.read_sql(query, conn)
        conn.close()
        if df.empty:
            raise ValueError("El DataFrame está vacío")
        logging.info(f"Datos cargados: {len(df)} registros")

        # Verificar las columnas disponibles
        logging.info(f"Columnas disponibles: {df.columns.tolist()}")

        # Manejo de valores nulos
        df['DESC_PORCENTAJE'] = df['DESC_PORCENTAJE'].fillna(df['DESC_PORCENTAJE'].median())
        df['MARGEN_GANANCIA_BRUTA_PORCENTAJE'] = df['MARGEN_GANANCIA_BRUTA_PORCENTAJE'].fillna(
            df['MARGEN_GANANCIA_BRUTA_PORCENTAJE'].median()
        )

        # Crear características temporales
        df['MES'] = pd.to_datetime(df['MES'])
        df['year'] = df['MES'].dt.year
        df['month'] = df['MES'].dt.month
        df['quarter'] = df['MES'].dt.quarter
        df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
        df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

        # Calcular promedios de DESC_PORCENTAJE y MARGEN_GANANCIA_BRUTA_PORCENTAJE por región y categoría
        region_col = next((col for col in df.columns if 'region' in col.lower()), 'nombre_region')
        categoria_col = next((col for col in df.columns if 'categoria' in col.lower()), 'categoria')
        producto_col = next((col for col in df.columns if 'nombre_producto' in col.lower()), 'nombre_producto')
        avg_metrics = df.groupby([region_col, categoria_col])[[
            'DESC_PORCENTAJE', 'MARGEN_GANANCIA_BRUTA_PORCENTAJE'
        ]].mean().reset_index()
        logging.info(f"Promedio DESC_PORCENTAJE: {avg_metrics['DESC_PORCENTAJE'].mean():.2f}%")
        logging.info(f"Promedio MARGEN_GANANCIA_BRUTA_PORCENTAJE: {avg_metrics['MARGEN_GANANCIA_BRUTA_PORCENTAJE'].mean():.2f}%")

        # Codificación de variables categóricas
        le_region = LabelEncoder()
        le_categoria = LabelEncoder()
        le_producto = LabelEncoder()
        df['region_encoded'] = le_region.fit_transform(df[region_col])
        df['categoria_encoded'] = le_categoria.fit_transform(df[categoria_col])
        df['producto_encoded'] = le_producto.fit_transform(df[producto_col])

        # Detectar dinámicamente la columna de cantidades vendidas
        target_col_candidates = [col for col in df.columns if any(keyword in col.lower() for keyword in ['m3', 'vendida','vendidos', 'unidades'])]
        target_col = target_col_candidates[0] if target_col_candidates else 'M3_VENDIDOS'
        logging.info(f"Variable objetivo detectada: {target_col} (candidatas: {target_col_candidates})")

        # Seleccionar características y variable objetivo
        features = [
            'year', 'month', 'quarter' , 'month_sin', 'month_cos',
            'region_encoded', 'categoria_encoded', 'producto_encoded',
            'DESC_PORCENTAJE', 'MARGEN_GANANCIA_BRUTA_PORCENTAJE'
        ]
        target = target_col

        return df, features, target, le_region, le_categoria, le_producto, avg_metrics
    except Exception as e:
        logging.error(f"Error en carga y preprocesamiento: {e}")
        raise

df, features, target, le_region, le_categoria, le_producto, avg_metrics = load_and_preprocess_data()
print(f"Datos cargados y preprocesados. Filas: {len(df)}, Variable objetivo: {target}")
df.head(5)

2025-06-12 19:15:29,894 - INFO - Conexión a Snowflake exitosa
2025-06-12 19:15:36,794 - INFO - Datos cargados: 5479 registros
2025-06-12 19:15:36,796 - INFO - Columnas disponibles: ['MES', 'NOMBRE_REGION', 'CATEGORIA', 'NOMBRE_PRODUCTO', 'REGION_CATEGORIA', 'TICKETS', 'CANTIDAD', 'PRECIO_PROMEDIO', 'VENTAS_TOTALES', 'VENTAS_BRUTAS', 'DESCUENTOS', 'DESC_PORCENTAJE', 'COSTOS', 'GANANCIA_BRUTA', 'MARGEN_GANANCIA_BRUTA_PORCENTAJE', 'M3_VENDIDOS']
2025-06-12 19:15:36,819 - INFO - Promedio DESC_PORCENTAJE: 8.36%
2025-06-12 19:15:36,821 - INFO - Promedio MARGEN_GANANCIA_BRUTA_PORCENTAJE: 49.20%
2025-06-12 19:15:36,860 - INFO - Variable objetivo detectada: M3_VENDIDOS (candidatas: ['M3_VENDIDOS'])


Datos cargados y preprocesados. Filas: 5479, Variable objetivo: M3_VENDIDOS


,MES,NOMBRE_REGION,CATEGORIA,NOMBRE_PRODUCTO,REGION_CATEGORIA,TICKETS,CANTIDAD,PRECIO_PROMEDIO,VENTAS_TOTALES,VENTAS_BRUTAS,...,MARGEN_GANANCIA_BRUTA_PORCENTAJE,M3_VENDIDOS,year,month,quarter,month_sin,month_cos,region_encoded,categoria_encoded,producto_encoded
0,2022-01-01,Antioquia,Jugo,Jugo Lima 2000mL x 12uds,Antioquia-Jugo,24,486,96000.0,40404480.0,46656000.0,...,36.490211,11.6640,2022,1,1,0.5,0.866025,0,4,6
1,2022-01-01,Antioquia,Jugo,Jugo Manzana 2000mL x 24uds,Antioquia-Jugo,18,312,192000.0,54581760.0,59904000.0,...,39.636978,14.9760,2022,1,1,0.5,0.866025,0,4,7
2,2022-01-01,Antioquia,Bebida Energética,Power Up 500mL x 6uds,Antioquia-Bebida Energética,20,382,30000.0,10404000.0,11460000.0,...,50.432526,1.1460,2022,1,1,0.5,0.866025,0,1,9
3,2022-01-01,Antioquia,Gaseosa,Cola 600mL x 24uds,Antioquia-Gaseosa,19,394,50400.0,18682272.0,19857600.0,...,45.335985,5.6736,2022,1,1,0.5,0.866025,0,3,4
4,2022-01-01,Antioquia,Agua,Agua Sin Gas 1L x 6uds,Antioquia-Agua,50,1068,15000.0,14099400.0,16020000.0,...,54.551258,6.4080,2022,1,1,0.5,0.866025,0,0,1


## 4. Selección de Características con RFE

Utilizamos RFE para seleccionar las características más importantes.

In [323]:
def select_features(df, features, target):
    try:
        X = df[features]
        y = df[target]
        if X.empty or y.empty:
            raise ValueError("Datos de entrada vacíos para RFE")

        # Escalar datos
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        # Modelo base para RFE y cálculo de importancias
        model = xgb.XGBRegressor(random_state=42)
        rfe = RFE(estimator=model, n_features_to_select=7)
        rfe.fit(X_scaled, y)

        # Obtener importancias de todas las features
        model.fit(X_scaled, y)  # Entrenar el modelo completo para obtener importancias
        importances = model.feature_importances_
        feature_importance_df = pd.DataFrame({
            'Feature': features,
            'Importance': importances
        }).sort_values(by='Importance', ascending=False)
        logging.info("Importancias de las features:")
        logging.info(feature_importance_df.to_string(index=False))

        # Seleccionar características
        selected_features = [f for f, s in zip(features, rfe.support_) if s]
        logging.info(f"Características seleccionadas: {selected_features}")

        return selected_features, scaler
    except Exception as e:
        logging.error(f"Error en selección de características: {e}")
        raise

selected_features, scaler = select_features(df, features, target)

2025-06-12 19:15:38,338 - INFO - Importancias de las features:
2025-06-12 19:15:38,345 - INFO -                          Feature  Importance
               categoria_encoded    0.446014
                  region_encoded    0.270611
                producto_encoded    0.218028
                            year    0.033012
                           month    0.007775
                 DESC_PORCENTAJE    0.007668
MARGEN_GANANCIA_BRUTA_PORCENTAJE    0.005850
                       month_sin    0.005826
                       month_cos    0.005215
                         quarter    0.000000
2025-06-12 19:15:38,348 - INFO - Características seleccionadas: ['year', 'month', 'region_encoded', 'categoria_encoded', 'producto_encoded', 'DESC_PORCENTAJE', 'MARGEN_GANANCIA_BRUTA_PORCENTAJE']


## 5. Partición de Datos

Dividimos los datos en 70% entrenamiento, 20% prueba y 10% validación, respetando la temporalidad.

In [324]:
def split_data(df, selected_features, target):
    try:
        if len(df) < 10:  # Mínimo para evitar divisiones inválidas
            raise ValueError("Demasiado pocos datos para partición")
        df = df.sort_values('MES')
        df = df.sample(frac=1, random_state=42).reset_index(drop=True)  # Mezclar datos
        X = df[selected_features]
        y = df[target]

        total_size = len(df)
        train_size = int(0.7 * total_size)
        test_size = int(0.2 * total_size)
        val_size = total_size - train_size - test_size

        if train_size <= 0 or test_size <= 0 or val_size <= 0:
            raise ValueError("Tamaños de partición inválidos")

        X_train = X.iloc[:train_size]
        y_train = y.iloc[:train_size]
        X_test = X.iloc[train_size:train_size + test_size]
        y_test = y.iloc[train_size:train_size + test_size]
        X_val = X.iloc[train_size + test_size:]
        y_val = y.iloc[train_size + test_size:]

        logging.info(f"Tamaño entrenamiento: {len(X_train)}, prueba: {len(X_test)}, validación: {len(X_val)}")
        return X_train, y_train, X_test, y_test, X_val, y_val
    except Exception as e:
        logging.error(f"Error en partición de datos: {e}")
        raise

X_train, y_train, X_test, y_test, X_val, y_val = split_data(df, selected_features, target)
print(f"Partición exitosa. Train: {len(X_train)}, Test: {len(X_test)}, Val: {len(X_val)}")

2025-06-12 19:15:38,414 - INFO - Tamaño entrenamiento: 3835, prueba: 1095, validación: 549


Partición exitosa. Train: 3835, Test: 1095, Val: 549


## 6. Entrenamiento del Modelo con RandomizedSearchCV

Optimizamos los hiperparámetros utilizando RandomizedSearchCV para mayor eficiencia.

In [325]:
def train_model(X_train, y_train):
    try:
        # Rangos de hiperparámetros optimizados para mitigar el sobreajuste
        param_dist_optimized = {
            'n_estimators': [50, 100, 200, 300, 400], # Se reduce el máximo un poco
            'max_depth': [3, 4, 5, 6, 7],           # Rango más conservador para la profundidad
            'learning_rate': [0.01, 0.03, 0.05, 0.1], # Se enfoca en learning rates más pequeños
            'subsample': [0.6, 0.7, 0.8, 0.9],      # Valores que fomentan la regularización
            'colsample_bytree': [0.6, 0.7, 0.8, 0.9], # Valores que fomentan la regularización
            'reg_lambda': [0.1, 1.0, 5.0, 10.0, 20.0], # Ampliamos el rango de regularización L2
            'reg_alpha': [0, 0.1, 0.5, 1.0, 5.0],      # Ampliamos el rango de regularización L1
            'min_child_weight': [1, 3, 5, 7, 10],   # Añadido: Importante para control de sobreajuste
            'gamma': [0, 0.1, 0.2, 0.3, 0.4]         # Añadido: Mínima reducción de pérdida para split
        }

        model = xgb.XGBRegressor(random_state=42, objective='reg:squarederror')
        
        # Ajustes a RandomizedSearchCV
        random_search = RandomizedSearchCV(
            model, 
            param_distributions=param_dist_optimized, 
            n_iter=50,  # Aumentado el número de iteraciones para explorar más combinaciones
            cv=5,       # Mantenemos 5-fold cross-validation, que es un buen balance
            scoring='neg_mean_squared_error', 
            n_jobs=-1,  # Usar todos los núcleos disponibles
            random_state=42,
            verbose=1   # Para ver el progreso durante la búsqueda
        )
        random_search.fit(X_train, y_train)

        best_model = random_search.best_estimator_
        logging.info(f"Mejores hiperparámetros: {random_search.best_params_}")
        logging.info(f"Mejor score de CV (neg_mean_squared_error): {random_search.best_score_}")

        return best_model
    except Exception as e:
        logging.error(f"Error en el entrenamiento del modelo: {e}")
        raise

model = train_model(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


2025-06-12 19:16:21,517 - INFO - Mejores hiperparámetros: {'subsample': 0.7, 'reg_lambda': 0.1, 'reg_alpha': 0, 'n_estimators': 200, 'min_child_weight': 10, 'max_depth': 7, 'learning_rate': 0.05, 'gamma': 0, 'colsample_bytree': 0.9}
2025-06-12 19:16:21,520 - INFO - Mejor score de CV (neg_mean_squared_error): -5.3088058411053085


## 7. Evaluación del Modelo

Calculamos métricas de rendimiento para entrenamiento, prueba y validación.

In [326]:
def evaluate_model(model, X_train, y_train, X_test, y_test, X_val, y_val):
    try:
        def calculate_metrics(y_true, y_pred):
            rmse = np.sqrt(mean_squared_error(y_true, y_pred))
            mae = mean_absolute_error(y_true, y_pred)
            smape = np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true) + 1e-10)) * 100
            r2 = r2_score(y_true, y_pred)
            return {'rmse': rmse, 'mae': mae, 'smape': smape, 'r2': r2}

        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)
        y_pred_val = model.predict(X_val)

        metrics_train = calculate_metrics(y_train, y_pred_train)
        metrics_test = calculate_metrics(y_test, y_pred_test)
        metrics_val = calculate_metrics(y_val, y_pred_val)

        metrics_summary = {
            'Train': metrics_train,
            'Test': metrics_test,
            'Validation': metrics_val
        }

        logging.info(f"Métricas de entrenamiento: {metrics_train}")
        logging.info(f"Métricas de prueba: {metrics_test}")
        logging.info(f"Métricas de validación: {metrics_val}")

        return metrics_summary
    except Exception as e:
        logging.error(f"Error en evaluación del modelo: {e}")
        raise

metrics_summary = evaluate_model(model, X_train, y_train, X_test, y_test, X_val, y_val)

2025-06-12 19:16:21,657 - INFO - Métricas de entrenamiento: {'rmse': 1.28231080958805, 'mae': 0.6492040057502212, 'smape': 37.603075119539746, 'r2': 0.9768723454438937}
2025-06-12 19:16:21,660 - INFO - Métricas de prueba: {'rmse': 2.568599333214191, 'mae': 1.1669882154360431, 'smape': 48.12261178219862, 'r2': 0.9217789121445349}
2025-06-12 19:16:21,662 - INFO - Métricas de validación: {'rmse': 2.509022548203736, 'mae': 1.117475462840381, 'smape': 47.40524683459827, 'r2': 0.9108132812333396}


## 8. Generar Predicciones Futuras

Predecimos las cantidades vendidas para los próximos 12 meses con intervalos de confianza del 95% por región y categoría.

In [327]:
def generate_future_predictions(df, model, selected_features, le_region, le_categoria, le_producto,
                              custom_desc_global=None, custom_margen_global=None,
                              custom_desc_dict=None, custom_margen_dict=None,
                              use_custom_values=False, months_to_avg=12):
    try:
        last_date = df['MES'].max()
        future_dates = pd.date_range(start=last_date + pd.offsets.MonthBegin(1), periods=12, freq='MS')

        # Detectar columnas dinámicamente
        region_col = next((col for col in df.columns if 'region' in col.lower()), 'nombre_region')
        categoria_col = next((col for col in df.columns if 'categoria' in col.lower()), 'categoria')
        producto_col = next((col for col in df.columns if 'nombre_producto' in col.lower()), 'nombre_producto')
        logging.info(f"Columnas detectadas: region={region_col}, categoria={categoria_col}, producto={producto_col}")

        # Filtrar últimos N meses para calcular promedios
        historical_cutoff = last_date - pd.DateOffset(months=months_to_avg)
        df_last12 = df[df['MES'] >= historical_cutoff]

        # Obtener combinaciones únicas
        combinations = df[[region_col, categoria_col, producto_col]].drop_duplicates()

        # DataFrame para almacenar promedios calculados
        promedios_data = []

        future_data = []
        for _, row in combinations.iterrows():
            region = row[region_col]
            categoria = row[categoria_col]
            producto = row[producto_col]

            # Calcular o asignar valores para descuentos y márgenes
            key = (region, categoria, producto)
            if use_custom_values:
                if custom_desc_global is not None:
                    desc_porcentaje = custom_desc_global
                    tipo_desc = 'Global'
                elif custom_desc_dict is not None and key in custom_desc_dict:
                    desc_porcentaje = custom_desc_dict[key]
                    tipo_desc = 'Personalizado'
                else:
                    mask = (df_last12[region_col] == region) & (df_last12[categoria_col] == categoria) & (df_last12[producto_col] == producto)
                    filtered = df_last12[mask]
                    desc_porcentaje = filtered['DESC_PORCENTAJE'].mean() if not filtered.empty else df_last12['DESC_PORCENTAJE'].median()
                    tipo_desc = 'Promedio'
                
                if custom_margen_global is not None:
                    margen_ganancia = custom_margen_global
                    tipo_margen = 'Global'
                elif custom_margen_dict is not None and key in custom_margen_dict:
                    margen_ganancia = custom_margen_dict[key]
                    tipo_margen = 'Personalizado'
                else:
                    mask = (df_last12[region_col] == region) & (df_last12[categoria_col] == categoria) & (df_last12[producto_col] == producto)
                    filtered = df_last12[mask]
                    margen_ganancia = filtered['MARGEN_GANANCIA_BRUTA_PORCENTAJE'].mean() if not filtered.empty else df_last12['MARGEN_GANANCIA_BRUTA_PORCENTAJE'].median()
                    tipo_margen = 'Promedio'
            else:
                mask = (df_last12[region_col] == region) & (df_last12[categoria_col] == categoria) & (df_last12[producto_col] == producto)
                filtered = df_last12[mask]
                desc_porcentaje = filtered['DESC_PORCENTAJE'].mean() if not filtered.empty else df_last12['DESC_PORCENTAJE'].median()
                margen_ganancia = filtered['MARGEN_GANANCIA_BRUTA_PORCENTAJE'].mean() if not filtered.empty else df_last12['MARGEN_GANANCIA_BRUTA_PORCENTAJE'].median()
                tipo_desc = tipo_margen = 'Promedio'
            
            # Guardar los valores usados para verificación
            promedios_data.append({
                region_col: region,
                categoria_col: categoria,
                producto_col: producto,
                'DESC_PORCENTAJE': desc_porcentaje,
                'MARGEN_GANANCIA_BRUTA_PORCENTAJE': margen_ganancia,
                'Tipo_Valor': f"{tipo_desc}/{tipo_margen}"
            })

            # Generar datos futuros para cada fecha
            for date in future_dates:
                future_data.append({
                    'MES': date,
                    region_col: region,
                    categoria_col: categoria,
                    producto_col: producto,
                    'year': date.year,
                    'month': date.month,
                    'quarter': (date.month - 1) // 3 + 1,
                    'month_sin': np.sin(2 * np.pi * date.month / 12),
                    'month_cos': np.cos(2 * np.pi * date.month / 12),
                    'region_encoded': le_region.transform([region])[0],
                    'categoria_encoded': le_categoria.transform([categoria])[0],
                    'producto_encoded': le_producto.transform([producto])[0],
                    'DESC_PORCENTAJE': desc_porcentaje,
                    'MARGEN_GANANCIA_BRUTA_PORCENTAJE': margen_ganancia
                })

        # Crear DataFrame de promedios
        promedios_df = pd.DataFrame(promedios_data)
        
        # Mostrar promedios calculados
        print("=== Promedios utilizados ===")
        print(promedios_df.to_string(index=False))
        print("\n")

        future_df = pd.DataFrame(future_data)
        X_future = future_df[selected_features]

        # Generar predicciones con intervalos de confianza
        n_iterations = 100
        preds = np.zeros((n_iterations, len(X_future)))
        
        for i in range(n_iterations):
            X_perturbed = X_future.copy()
            for col in X_perturbed.columns:
                if col in ['region_encoded', 'categoria_encoded', 'producto_encoded']:
                    continue  # No perturbar columnas codificadas
                if np.issubdtype(X_perturbed[col].dtype, np.number):
                    X_perturbed[col] = X_perturbed[col] * np.random.normal(1, 0.01, len(X_perturbed))
            
            preds[i] = model.predict(X_perturbed)

        y_pred = np.mean(preds, axis=0)
        ci_lower = np.percentile(preds, 2.5, axis=0)
        ci_upper = np.percentile(preds, 97.5, axis=0)

        future_df['predicted_quantities'] = y_pred
        future_df['ci_lower'] = ci_lower
        future_df['ci_upper'] = ci_upper

        return future_df, promedios_df

    except Exception as e:
        logging.error(f"Error en generación de predicciones futuras: {e}")
        raise

# Ejemplos de uso:
# 1. Usar promedios históricos (default)
future_predictions, promedios = generate_future_predictions(df, model, selected_features, le_region, le_categoria, le_producto)

# 2. Usar valores globales para todos
future_predictions, promedios = generate_future_predictions(
    df, model, selected_features, le_region, le_categoria, le_producto,
    custom_desc_global=0.15,  # 15% descuento para todos
    custom_margen_global=0.25,  # 25% margen para todos
    use_custom_values=True
)

# 3. Usar valores específicos para algunas combinaciones
custom_desc = {('Región A', 'Categoría X', 'Producto 1'): 0.10, ('Región B', 'Categoría Y', 'Producto 2'): 0.20}
custom_margen = {('Región A', 'Categoría X', 'Producto 1'): 0.30, ('Región B', 'Categoría Y', 'Producto 2'): 0.35}
future_predictions, promedios = generate_future_predictions(
    df, model, selected_features, le_region, le_categoria, le_producto,
    custom_desc_dict=custom_desc,
    custom_margen_dict=custom_margen,
    use_custom_values=True
)

2025-06-12 19:16:21,834 - INFO - Columnas detectadas: region=NOMBRE_REGION, categoria=CATEGORIA, producto=NOMBRE_PRODUCTO


=== Promedios utilizados ===
     NOMBRE_REGION         CATEGORIA             NOMBRE_PRODUCTO  DESC_PORCENTAJE  MARGEN_GANANCIA_BRUTA_PORCENTAJE        Tipo_Valor
         Antioquia              Jugo    Jugo Lima 2000mL x 12uds         6.930806                         45.680861 Promedio/Promedio
         Antioquia              Jugo Jugo Manzana 2000mL x 24uds         6.160303                         47.673493 Promedio/Promedio
         Antioquia Bebida Energética       Power Up 500mL x 6uds         6.648986                         54.712551 Promedio/Promedio
         Antioquia           Gaseosa          Cola 600mL x 24uds         6.422126                         51.985619 Promedio/Promedio
         Antioquia              Agua      Agua Sin Gas 1L x 6uds         7.400447                         60.708352 Promedio/Promedio
         Antioquia              Jugo     Jugo Lima 1000mL x 6uds         7.090366                         39.194765 Promedio/Promedio
         Antioquia      Bebida de

2025-06-12 19:16:26,913 - INFO - Columnas detectadas: region=NOMBRE_REGION, categoria=CATEGORIA, producto=NOMBRE_PRODUCTO


=== Promedios utilizados ===
     NOMBRE_REGION         CATEGORIA             NOMBRE_PRODUCTO  DESC_PORCENTAJE  MARGEN_GANANCIA_BRUTA_PORCENTAJE    Tipo_Valor
         Antioquia              Jugo    Jugo Lima 2000mL x 12uds             0.15                              0.25 Global/Global
         Antioquia              Jugo Jugo Manzana 2000mL x 24uds             0.15                              0.25 Global/Global
         Antioquia Bebida Energética       Power Up 500mL x 6uds             0.15                              0.25 Global/Global
         Antioquia           Gaseosa          Cola 600mL x 24uds             0.15                              0.25 Global/Global
         Antioquia              Agua      Agua Sin Gas 1L x 6uds             0.15                              0.25 Global/Global
         Antioquia              Jugo     Jugo Lima 1000mL x 6uds             0.15                              0.25 Global/Global
         Antioquia      Bebida de Té       Té Verde 300mL x 6

2025-06-12 19:16:31,035 - INFO - Columnas detectadas: region=NOMBRE_REGION, categoria=CATEGORIA, producto=NOMBRE_PRODUCTO


=== Promedios utilizados ===
     NOMBRE_REGION         CATEGORIA             NOMBRE_PRODUCTO  DESC_PORCENTAJE  MARGEN_GANANCIA_BRUTA_PORCENTAJE        Tipo_Valor
         Antioquia              Jugo    Jugo Lima 2000mL x 12uds         6.930806                         45.680861 Promedio/Promedio
         Antioquia              Jugo Jugo Manzana 2000mL x 24uds         6.160303                         47.673493 Promedio/Promedio
         Antioquia Bebida Energética       Power Up 500mL x 6uds         6.648986                         54.712551 Promedio/Promedio
         Antioquia           Gaseosa          Cola 600mL x 24uds         6.422126                         51.985619 Promedio/Promedio
         Antioquia              Agua      Agua Sin Gas 1L x 6uds         7.400447                         60.708352 Promedio/Promedio
         Antioquia              Jugo     Jugo Lima 1000mL x 6uds         7.090366                         39.194765 Promedio/Promedio
         Antioquia      Bebida de

### PREDICCIONES

In [328]:
future_predictions[(future_predictions['NOMBRE_REGION']=='Meta') & (future_predictions['CATEGORIA']=='Bebida Energética')]

,MES,NOMBRE_REGION,CATEGORIA,NOMBRE_PRODUCTO,year,month,quarter,month_sin,month_cos,region_encoded,categoria_encoded,producto_encoded,DESC_PORCENTAJE,MARGEN_GANANCIA_BRUTA_PORCENTAJE,predicted_quantities,ci_lower,ci_upper
912,2025-06-01,Meta,Bebida Energética,Power Up 500mL x 6uds,2025,6,2,1.224647e-16,-1.000000e+00,5,1,9,6.438996,54.707375,0.147526,0.127721,0.198929
913,2025-07-01,Meta,Bebida Energética,Power Up 500mL x 6uds,2025,7,3,-5.000000e-01,-8.660254e-01,5,1,9,6.438996,54.707375,0.155618,0.130256,0.204312
914,2025-08-01,Meta,Bebida Energética,Power Up 500mL x 6uds,2025,8,3,-8.660254e-01,-5.000000e-01,5,1,9,6.438996,54.707375,0.173132,0.135624,0.218431
915,2025-09-01,Meta,Bebida Energética,Power Up 500mL x 6uds,2025,9,3,-1.000000e+00,-1.836970e-16,5,1,9,6.438996,54.707375,0.159219,0.107080,0.215252
916,2025-10-01,Meta,Bebida Energética,Power Up 500mL x 6uds,2025,10,4,-8.660254e-01,5.000000e-01,5,1,9,6.438996,54.707375,0.133320,0.091704,0.183168
917,2025-11-01,Meta,Bebida Energética,Power Up 500mL x 6uds,2025,11,4,-5.000000e-01,8.660254e-01,5,1,9,6.438996,54.707375,0.123578,0.058128,0.191756
918,2025-12-01,Meta,Bebida Energética,Power Up 500mL x 6uds,2025,12,4,-2.449294e-16,1.000000e+00,5,1,9,6.438996,54.707375,0.163049,0.058128,0.282696
919,2026-01-01,Meta,Bebida Energética,Power Up 500mL x 6uds,2026,1,1,5.000000e-01,8.660254e-01,5,1,9,6.438996,54.707375,0.104081,0.072299,0.154389
920,2026-02-01,Meta,Bebida Energética,Power Up 500mL x 6uds,2026,2,1,8.660254e-01,5.000000e-01,5,1,9,6.438996,54.707375,0.104637,0.075392,0.141404
921,2026-03-01,Meta,Bebida Energética,Power Up 500mL x 6uds,2026,3,1,1.000000e+00,6.123234e-17,5,1,9,6.438996,54.707375,0.134634,0.079619,0.184098


## 9. Visualización de Resultados

Generamos gráficas interactivas con Plotly para datos históricos y predicciones por región y categoría.

In [329]:
def plot_predictions(df, future_predictions):
    try:
        # Verificación inicial de columnas
        required_pred_cols = ['predicted_quantities', 'ci_lower', 'ci_upper']
        if not all(col in future_predictions.columns for col in required_pred_cols):
            raise ValueError("El DataFrame de predicciones no tiene la estructura esperada")
        
        # Detección automática de columnas
        region_col = next((col for col in df.columns if 'region' in col.lower()), 'NOMBRE_REGION')
        categoria_col = next((col for col in df.columns if 'categoria' in col.lower()), 'CATEGORIA')
        target_col = next((col for col in df.columns if 'm3' in col.lower()), 'M3_VENDIDOS')
        
        if target_col is None:
            raise ValueError("No se pudo detectar la columna objetivo. Verifique los nombres de columnas")

        # Feature engineering: añadir variables lagged
        df = df.sort_values('MES')
        df['lag_1'] = df.groupby([region_col, categoria_col])[target_col].shift(1)
        df['lag_2'] = df.groupby([region_col, categoria_col])[target_col].shift(2)
        df['lag_1'].fillna(df[target_col].mean(), inplace=True)
        df['lag_2'].fillna(df[target_col].mean(), inplace=True)
        selected_features.append('lag_1')
        selected_features.append('lag_2')

        # Obtener las últimas 24 meses de datos históricos y agrupar
        last_date = df['MES'].max()
        one_year_ago = last_date - pd.offsets.MonthBegin(24)
        hist_data = df[df['MES'] >= one_year_ago].groupby([region_col, categoria_col, 'MES'])[target_col].sum().reset_index()
        hist_data['type'] = 'Histórico'
        hist_data = hist_data.rename(columns={target_col: 'value', 'MES': 'prediction_date'})

        # Preparar datos de predicciones y agrupar
        pred_data = future_predictions.groupby([region_col, categoria_col, 'MES'])[['predicted_quantities', 'ci_lower', 'ci_upper']].sum().reset_index()
        pred_data['type'] = 'Pronóstico'
        pred_data = pred_data.rename(columns={
            'MES': 'prediction_date',
            'predicted_quantities': 'value',
            region_col: region_col,
            categoria_col: categoria_col
        })

        # Combinar datos históricos y predicciones
        prediction_combined = pd.concat([
            hist_data[['prediction_date', region_col, categoria_col, 'value', 'type']],
            pred_data[['prediction_date', region_col, categoria_col, 'value', 'type', 'ci_lower', 'ci_upper']]
        ]).reset_index(drop=True)

        # Obtener combinaciones únicas
        combinations = df[[region_col, categoria_col]].drop_duplicates()

        for _, (region, categoria) in enumerate(combinations.itertuples(index=False), 1):
            # Filtrar datos para esta combinación
            mask = ((prediction_combined[region_col] == region) & 
                    (prediction_combined[categoria_col] == categoria))
            plot_data = prediction_combined[mask].sort_values('prediction_date')

            if plot_data.empty:
                logging.warning(f"No hay datos para {region} - {categoria}")
                continue

            # Calcular línea de tendencia con regresión polinómica de grado 2
            hist_full = df[(df[region_col] == region) & 
                          (df[categoria_col] == categoria)].sort_values('MES')
            hist_full = hist_full.groupby(['MES'])[target_col].sum().reset_index()
            hist_full['days'] = (hist_full['MES'] - hist_full['MES'].min()).dt.days
            coefficients = np.polyfit(hist_full['days'], hist_full[target_col], 2)
            polynomial = np.poly1d(coefficients)

            # Extender tendencia al período completo
            all_dates = plot_data['prediction_date'].sort_values()
            all_dates_df = pd.DataFrame({'prediction_date': all_dates})
            all_dates_df['days'] = (all_dates_df['prediction_date'] - hist_full['MES'].min()).dt.days
            trend_values = polynomial(all_dates_df['days'])

            # Crear figura
            fig = go.Figure()

            # 1. Datos históricos (últimos 24 meses)
            hist_plot_data = plot_data[plot_data['type'] == 'Histórico']
            fig.add_trace(go.Scatter(
                x=hist_plot_data['prediction_date'],
                y=hist_plot_data['value'].round(2),
                mode='lines+markers',
                name='Histórico',
                line=dict(color='blue', width=1.5),
                hovertemplate='Fecha: %{x|%b %Y}<br>Histórico: %{y:.2f}<extra></extra>'
            ))

            # 2. Pronósticos
            pred_plot_data = plot_data[plot_data['type'] == 'Pronóstico']
            fig.add_trace(go.Scatter(
                x=pred_plot_data['prediction_date'],
                y=pred_plot_data['value'].round(2),
                mode='lines+markers',
                name='Pronóstico',
                line=dict(color='red', width=2),
                hovertemplate='Fecha: %{x|%b %Y}<br>Pronóstico: %{y:.2f}<extra></extra>'
            ))

            # 3. Intervalo de confianza
            fig.add_trace(go.Scatter(
                x=pred_plot_data['prediction_date'],
                y=pred_plot_data['ci_upper'].round(2),
                mode='lines',
                line=dict(color='rgba(255,0,0,0.3)', width=1, dash='dash'),
                name='IC 95% Superior',
                showlegend=False
            ))
            fig.add_trace(go.Scatter(
                x=pred_plot_data['prediction_date'],
                y=pred_plot_data['ci_lower'].round(2),
                mode='lines',
                line=dict(color='rgba(255,0,0,0.3)', width=1, dash='dash'),
                fill='tonexty',
                fillcolor='rgba(255,0,0,0.1)',
                name='IC 95% Inferior',
                showlegend=True
            ))

            # 4. Línea de tendencia (polinómica de grado 2)
            fig.add_trace(go.Scatter(
                x=all_dates_df['prediction_date'],
                y=trend_values.round(2),
                mode='lines',
                name='Tendencia',
                line=dict(color='green', width=2, dash='dot'),
                hovertemplate='Tendencia: %{y:.2f}<extra></extra>'
            ))

            # Línea vertical separadora
            last_historical_date = hist_data['prediction_date'].max()
            fig.add_vline(
                x=last_historical_date.timestamp() * 1000,
                line_width=2,
                line_dash="dash",
                line_color="gray",
                annotation_text="Fin histórico",
                annotation_position="top right"
            )

            # Configuración del layout
            fig.update_layout(
                title=f'{region} - {categoria}',
                xaxis_title='Fecha',
                yaxis_title='Cantidad Vendida (m³)',
                hovermode='x unified',
                height=500,
                width=900,
                legend=dict(
                    orientation="h",
                    yanchor="bottom",
                    y=1.02,
                    xanchor="right",
                    x=1
                ),
                margin=dict(t=50)
            )

            fig.show()

    except Exception as e:
        logging.error(f"Error en visualización: {e}")
        raise

# Uso:
plot_predictions(df, future_predictions)

## 10. Almacenar Resultados

Guardamos las predicciones y métricas en Snowflake utilizando `COPY INTO` en las predicciones e `INSERT INTO` para las métricas

In [332]:
from pytz import timezone
import os
import joblib
from datetime import datetime
from tqdm import tqdm


def save_results(model, future_predictions, metrics_summary):
    conn = None
    # Definimos la ruta temporal para el archivo CSV
    temp_csv_path = 'temp_predictions.csv'
    try:
        # 1. Guardar modelo localmente
        os.makedirs('Modelos_Entrenados', exist_ok=True)
        timestamp = datetime.now(timezone('America/Bogota'))
        timestamp_str = timestamp.strftime('%Y-%m-%d %H:%M:%S')

        model_path = f'Modelos_Entrenados/sales_forecast_model_xgboost_{timestamp.strftime("%Y%m%d_%H%M")}.pkl'

        with tqdm(total=1, desc="Guardando modelo localmente") as pbar:
            joblib.dump(model, model_path)
            pbar.update(1)
            logging.info(f"Modelo guardado en: {model_path}")

        # 2. Conexión a Snowflake
        with tqdm(total=1, desc="Conectando a Snowflake") as pbar:
            conn = get_snowflake_connection() # Asegúrate de que esta función esté implementada
            cursor = conn.cursor()
            pbar.update(1)

        # 3. Configurar base de datos y tablas
        setup_steps = [
            ("Usando base de datos", "USE DATABASE BEBIDAS_PROJECT"),

            ("Eliminando tabla de predicciones si existe", "DROP TABLE IF EXISTS BEBIDAS_PROJECT.BEBIDAS_ANALYTICS.SALES_PREDICTIONS"),
            ("Creando tabla de predicciones", """
                CREATE TABLE BEBIDAS_PROJECT.BEBIDAS_ANALYTICS.SALES_PREDICTIONS (
                    ID INTEGER AUTOINCREMENT ,
                    TIMESTAMP_COL TIMESTAMP_NTZ,
                    PREDICTION_DATE DATE,
                    REGION VARCHAR(100),
                    CATEGORIA VARCHAR(100),
                    PREDICTED_QUANTITIES FLOAT,
                    CI_LOWER FLOAT,
                    CI_UPPER FLOAT
                )
            """),

            ("Creando tabla de métricas (global si no existe)", """
                CREATE TABLE IF NOT EXISTS BEBIDAS_PROJECT.BEBIDAS_ANALYTICS.ML_MODEL_METRICS_SALES_PREDICTIONS_XGBOOST (
                    ID INTEGER AUTOINCREMENT,
                    TIMESTAMP_COL TIMESTAMP_NTZ,
                    DATASET VARCHAR(50),
                    RMSE FLOAT,
                    MAE FLOAT,
                    SMAPE FLOAT,
                    R2 FLOAT,
                    MODEL_QUALITY VARCHAR(20)
                )
            """)
        ]

        for desc, query in tqdm(setup_steps, desc="Configurando Snowflake"):
            cursor.execute(query)

        # --- ESTRATEGIA PARA PREDICCIONES: TRUNCATE y COPY INTO ---

        # 4. Preparar datos para COPY INTO (Crear el DataFrame para el CSV)
        with tqdm(total=1, desc="Preparando datos para COPY INTO") as pbar:
            df_to_copy = future_predictions.groupby(['MES', 'NOMBRE_REGION', 'CATEGORIA'])[
                ['predicted_quantities', 'ci_lower', 'ci_upper']
            ].sum().reset_index()

            df_to_copy['TIMESTAMP_COL'] = timestamp_str

            df_to_copy = df_to_copy.rename(columns={
                'MES': 'PREDICTION_DATE',
                'NOMBRE_REGION': 'REGION',
                'CATEGORIA': 'CATEGORIA',
                'predicted_quantities': 'PREDICTED_QUANTITIES',
                'ci_lower': 'CI_LOWER',
                'ci_upper': 'CI_UPPER'
            })

            df_to_copy = df_to_copy[[
                'TIMESTAMP_COL',
                'PREDICTION_DATE',
                'REGION',
                'CATEGORIA',
                'PREDICTED_QUANTITIES',
                'CI_LOWER',
                'CI_UPPER'
            ]]

            df_to_copy.to_csv(temp_csv_path, index=False, header=False, encoding='utf-8')
            pbar.update(1)
            logging.info(f"Datos de predicciones guardados en CSV temporal: {temp_csv_path}")

        # 5. Subir el archivo CSV al stage de usuario de Snowflake y luego COPY INTO
        with tqdm(total=1, desc="Subiendo CSV a Snowflake y copiando datos") as pbar:
            cursor.execute(f"PUT file://{temp_csv_path} @~/predictions_stage AUTO_COMPRESS = TRUE OVERWRITE = TRUE")
            logging.info(f"Archivo {temp_csv_path} subido a Snowflake stage.")

            copy_query = f"""
                COPY INTO BEBIDAS_PROJECT.BEBIDAS_ANALYTICS.SALES_PREDICTIONS (TIMESTAMP_COL, PREDICTION_DATE, REGION, CATEGORIA, PREDICTED_QUANTITIES, CI_LOWER, CI_UPPER)
                FROM @~/predictions_stage/{os.path.basename(temp_csv_path)}.gz
                FILE_FORMAT = (TYPE = 'CSV' FIELD_DELIMITER = ',' SKIP_HEADER = 0 FIELD_OPTIONALLY_ENCLOSED_BY = '\"')
            """
            cursor.execute(copy_query)
            logging.info("Datos copiados a SALES_PREDICTIONS desde el stage.")
            pbar.update(1)

        # --- Insertar métricas para Train, Test y Validation ---
        insert_metrics_query = """
            INSERT INTO BEBIDAS_PROJECT.BEBIDAS_ANALYTICS.ML_MODEL_METRICS_SALES_PREDICTIONS_XGBOOST
            (TIMESTAMP_COL, DATASET, RMSE, MAE, SMAPE, R2, MODEL_QUALITY)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """

        datasets_to_insert = ['Train', 'Test', 'Validation']

        with tqdm(total=len(datasets_to_insert), desc="Insertando métricas por dataset") as pbar:
            for dataset_name in datasets_to_insert:
                metrics = metrics_summary.get(dataset_name, {})
                rmse_value = float(metrics.get('rmse', 0))
                mae_value = float(metrics.get('mae', 0))
                smape_value = float(metrics.get('smape', 0))
                r2_value = float(metrics.get('r2', 0))

                model_quality = 'Bueno' if r2_value >= 0.8 else 'Aceptable' if r2_value >= 0.5 else 'Pobre'

                cursor.execute(insert_metrics_query, (
                    timestamp_str,
                    dataset_name.lower(),
                    rmse_value,
                    mae_value,
                    smape_value,
                    r2_value,
                    model_quality
                ))
                pbar.update(1)
                logging.info(f"Métricas para '{dataset_name}' insertadas en Snowflake.")

        # 8. Confirmar transacción y mostrar resumen
        with tqdm(total=1, desc="Finalizando") as pbar:
            conn.commit()
            logging.info("Datos guardados exitosamente en Snowflake")

            cursor.execute(f"SELECT COUNT(*) FROM BEBIDAS_PROJECT.BEBIDAS_ANALYTICS.SALES_PREDICTIONS")
            pred_count = cursor.fetchone()[0]

            cursor.execute(f"SELECT COUNT(*) FROM BEBIDAS_PROJECT.BEBIDAS_ANALYTICS.ML_MODEL_METRICS_SALES_PREDICTIONS_XGBOOST")
            metrics_count = cursor.fetchone()[0]

            print(f"\nResumen:")
            print(f"   - Modelo guardado en: {model_path}")
            print(f"   - Predicciones insertadas: {pred_count}")
            print(f"   - Métricas globales insertadas: {metrics_count}")
            pbar.update(1)

    except Exception as e:
        if conn:
            conn.rollback()
        logging.error(f"Error al guardar resultados: {str(e)}")
        raise
    finally:
        if conn:
            conn.close()
        if os.path.exists(temp_csv_path):
            os.remove(temp_csv_path)
            logging.info(f"Archivo temporal {temp_csv_path} eliminado.")

# Uso
save_results(model, future_predictions, metrics_summary)

Guardando modelo localmente:   0%|          | 0/1 [00:00<?, ?it/s]

2025-06-12 19:21:04,593 - INFO - Modelo guardado en: Modelos_Entrenados/sales_forecast_model_xgboost_20250612_1921.pkl
Subiendo CSV a Snowflake y copiando datos:   0%|          | 0/1 [00:00<?, ?it/s]2025-06-12 19:21:08,460 - INFO - Archivo temp_predictions.csv subido a Snowflake stage.
2025-06-12 19:21:09,442 - INFO - Datos copiados a SALES_PREDICTIONS desde el stage.
Finalizando: 100%|██████████| 1/1 [00:00<00:00,  1.40it/s]


Resumen:
   - Modelo guardado en: Modelos_Entrenados/sales_forecast_model_xgboost_20250612_1921.pkl
   - Predicciones insertadas: 600
   - Métricas globales insertadas: 3



2025-06-12 19:21:12,417 - INFO - Archivo temporal temp_predictions.csv eliminado.
